In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ete3

from src.phylogenetic_signal import PagelsLambda
from src.ihmp import get_diffs
from src.greengenes import (
    aggregate_otu_table,
    generate_otu_keys,
    aggregate_tree,
)

In [2]:
data = {
    name: get_diffs(name, get_abundances=True, log=False) # undo log transform
    for name in ["ibd", "moms", "t2d"]
}


In [3]:
%%bash

# Download greengenes OTUs dataset, if necessary
if ! [ -d greengenes/data/gg_13_5_otus/ ]; then
    if ! [ -f greengenes/data/gg_13_5_otus.tar.gz ]; then
        wget -O greengenes/data/gg_13_5_otus.tar.gz \
            https://gg-sg-web.s3-us-west-2.amazonaws.com/downloads/greengenes_database/gg_13_5/gg_13_5_otus.tar.gz
    fi
    tar -xvf greengenes/data/gg_13_5_otus.tar.gz -C greengenes/data/
fi

In [4]:
results = pd.DataFrame(columns=["dataset", "cutoff", "sample", "lambda"])
# pl = PagelsLambda(
#     tree="/home/phil/phylosig/greengenes/data/gg_13_5_otus_99_annotated.tree",
#     format=1,
#     quoted_node_names=True,
# )

# for cutoff in (99, 97, 94, 91, 88, 85):
#     for dataset, df in data.items():
#         otu_keys = generate_otu_keys(cutoff)
#         otu_table = aggregate_otu_table(df, otu_keys)
#         cov = aggregate_covariance_matrix(df.cov(), otu_keys)
#         cov_df = make_cov_df(cov, otu_table.index)
#         for sample in otu_table.columns:
#             pl = PagelsLambda(otu_table[sample], cov_df)
#             results = results.append(
#                 {
#                     "dataset": dataset,
#                     "cutoff": cutoff,
#                     "sample": sample,
#                     "lambda": pl.lambda_,
#                 },
#                 ignore_index=True,
#             )

In [5]:
# master_tree = ete3.Tree(
#     "/home/phil/phylosig/greengenes/data/gg_13_5_otus_99_annotated.tree",
#     format=1,
#     quoted_node_names=True,
# )
for x in [99, 97, 94, 91, 88]:
    mapping, leaves, tree = generate_otu_keys(x)
    # tree = ete3.Tree(
    #     f"/home/phil/phylosig/greengenes/data/gg_13_5_otus/trees/{x}_otus.tree",
    #     format=1,
    #     quoted_node_names=True,
    # )

    for name, df in data.items():
        otu_table = aggregate_otu_table(df, mapping)

        # Restrict tree to OTUs in the OTU table
        dataset_tree = tree.copy()
        dataset_tree.prune(otu_table.index)

        for sample in otu_table.columns:
            pl = PagelsLambda(otu_table[sample], dataset_tree)
            results = results.append(
                {
                    "dataset": name,
                    "cutoff": x,
                    "sample": sample,
                    "lambda": pl.lam,
                },
                ignore_index=True,
            )
            print(f"{name} {x} {sample} {pl.lam}")

TypeError: aggregate_otu_table() missing 1 required positional argument: 'rep2otus'

In [ ]:
otu_table = aggregate_otu_table(data["ibd"], mapping)
otu_table.sum(axis=1)


100%|██████████| 1425/1425 [00:01<00:00, 1228.87it/s]


site   patient  visit  sample  
feces  3001     4      CSM5FZ3N    1.006490
       3002     5      CSM5FZ3X    1.010096
                6      CSM5FZ3Z    1.013085
                8      CSM5FZ44    1.049808
                9      CSM5FZ46    1.010699
                                     ...   
       2021     11     MSM5LLIO    1.009499
       2026     4      MSM5LLIQ    1.004635
       2027     4      MSM5LLIS    1.000000
       2014     9      MSM5ZOJY    1.060451
                11     MSM633FF    1.006207
Length: 98, dtype: float64

In [ ]:
# Is there redundancy in the mapping?
otu2rep, rep2otus, leaves, tree = generate_otu_keys(99)

otu2rep

{'42435': {'228054'},
 '228054': {'228054'},
 '844608': {'228054'},
 '178780': {'553316'},
 '198479': {'4466751'},
 '187280': {'4368610'},
 '179180': {'196144'},
 '175058': {'792673'},
 '176884': {'300026'},
 '83889': {'228057'},
 '228057': {'228057'},
 '234102': {'228057'},
 '234685': {'228057'},
 '1121497': {'228057'},
 '767731': {'228057'},
 '230047': {'228057'},
 '330751': {'228057'},
 '317400': {'228057'},
 '347564': {'228057'},
 '352714': {'228057'},
 '234168': {'228057'},
 '231859': {'228057'},
 '232604': {'228057'},
 '233538': {'228057'},
 '573838': {'228057'},
 '136068': {'313601'},
 '585338': {'228057'},
 '4474077': {'228057'},
 '1121583': {'228057'},
 '4342576': {'228057'},
 '4382430': {'228057'},
 '4486293': {'228057'},
 '192805': {'73627'},
 '73627': {'578985', '73627'},
 '785154': {'73627'},
 '581446': {'73627'},
 '177511': {'4412213'},
 '245190': {'73627'},
 '154341': {'154341'},
 '378462': {'154341', '378462'},
 '398771': {'154341', '378462'},
 '445143': {'154341', '378

In [ ]:
# First verify that leaves match
tree99 = ete3.Tree(
    f"/home/phil/phylosig/greengenes/data/gg_13_5_otus/trees/{x}_otus.tree",
    format=1,
    quoted_node_names=True,
)

tree99_leafset = set([x.name for x in tree99.get_leaves()])
print("Tree", len(tree99_leafset))

map99, keys99, tree99_clone = generate_otu_keys(99)
keys99_leafset = set(keys99)
print("Keys", len(keys99_leafset))
print("Intersection", len(keys99_leafset & tree99_leafset))

# Then check OTU table aggregation
otu_original = data["ibd"]
otu_aggregated = aggregate_otu_table(otu_original, map99)
print(otu_original.shape)
otu_leafset = set(otu_original.columns)
print("Original", len(otu_leafset))
print("Aggregated", len(otu_aggregated.index))
print("Intersection", len(otu_leafset & keys99_leafset))

In [ ]:
# Aggregate rows
original = data["ibd"]
otu2rep, rep2otus, tree = generate_otu_keys(97)

out_df = pd.DataFrame(
    index=original.index, 
    columns=[x for x in rep2otus if x in original.columns],
    data=0
)
otu2rep = {k: v for k, v in otu2rep.items() if k in original.columns}
original_otu_set = set(original.columns)

for otu, reps in otu2rep.items():
    # Pick 1st rep
    reps = original_otu_set & reps
    if len(reps) == 0:
        print(f"OTU {otu} has no representatives in the original table...")
        continue
    else:
        rep = reps.pop()
        out_df[rep] += original[otu]
# rep2otus = {k: v for k, v in rep2otus.items() if k in otu_original.columns}
# for otu, vals in map99.items():
#     if vals is None:
#         continue  # Skip OTUs which are not the first in their group
#     elif otu not in otu_original.columns:
#         continue
#     else:
#         vals_in_table = [val for val in vals if val in otu_original.columns]
#         out[otu] = np.sum(otu_original[vals_in_table], axis=1)

# out
out_df

548503    358944    535375  42256    348703  \
site  patient visit sample                                                    
feces 3001    4     CSM5FZ3N  0.000003  0.000152  0.000536    0.0  0.000000   
      3002    5     CSM5FZ3X  0.000000  0.000154  0.020679    0.0  0.000003   
              6     CSM5FZ3Z  0.000000  0.000872  0.037990    0.0  0.000004   
              8     CSM5FZ44  0.000000  0.003831  0.053640    0.0  0.000000   
              9     CSM5FZ46  0.000000  0.000649  0.035499    0.0  0.000000   
...                                ...       ...       ...    ...       ...   
      2021    11    MSM5LLIO  0.000000  0.000040  0.039392    0.0  0.000014   
      2026    4     MSM5LLIQ  0.000000  0.000678  0.003657    0.0  0.000019   
      2027    4     MSM5LLIS  0.000000  0.000000  0.000000    0.0  0.000000   
      2014    9     MSM5ZOJY  0.000002  0.000004  0.000469    0.0  0.000000   
              11    MSM633FF  0.000000  0.000000  0.000403    0.0  0.000000   

                                628226  196271  214031  208790  548556  ...  \
site  patient visit sample                                              ...   
feces 3001    4     CSM5FZ3N  0.000030     0.0     0.0     0.0     0.0  ...   
      3002    5     CSM5FZ3X  0.000157     0.0     0.0     0.0     0.0  ...   
              6     CSM5FZ3Z  0.000012     0.0     0.0     0.0     0.0  ...   
              8     CSM5FZ44  0.000000     0.0     0.0     0.0     0.0  ...   
              9     CSM5FZ46  0.000522     0.0     0.0     0.0     0.0  ...   
...                                ...     ...     ...     ...     ...  ...   
      2021    11    MSM5LLIO  0.000024     0.0     0.0     0.0     0.0  ...   
      2026    4     MSM5LLIQ  0.002463     0.0     0.0     0.0     0.0  ...   
      2027    4     MSM5LLIS  0.000000     0.0     0.0     0.0     0.0  ...   
      2014    9     MSM5ZOJY  0.007958     0.0     0.0     0.0     0.0  ...   
              11    MSM633FF  0.000645     0.0     0.0     0.0     0.0  ...   

                              350188   1974536    191222   353012  4432435  \
site  patient visit sample                                                   
feces 3001    4     CSM5FZ3N     0.0  0.000000  0.000429  0.00000      0.0   
      3002    5     CSM5FZ3X     0.0  0.000000  0.000012  0.00000      0.0   
              6     CSM5FZ3Z     0.0  0.000000  0.000000  0.00000      0.0   
              8     CSM5FZ44     0.0  0.000000  0.000000  0.00000      0.0   
              9     CSM5FZ46     0.0  0.000000  0.000000  0.00000      0.0   
...                              ...       ...       ...      ...      ...   
      2021    11    MSM5LLIO     0.0  0.000012  0.000024  0.00000      0.0   
      2026    4     MSM5LLIQ     0.0  0.000000  0.000000  0.00000      0.0   
      2027    4     MSM5LLIS     0.0  0.000000  0.000000  0.00000      0.0   
      2014    9     MSM5ZOJY     0.0  0.000000  0.000002  0.00000      0.0   
              11    MSM633FF     0.0  0.000000  0.000484  0.00129      0.0   

                                274619    589277   3304796    519000  609964  
site  patient visit sample                                                    
feces 3001    4     CSM5FZ3N  0.000003  0.532276  0.000039  0.000000     0.0  
      3002    5     CSM5FZ3X  0.000000  0.439626  0.000045  0.000000     0.0  
              6     CSM5FZ3Z  0.000000  0.415302  0.000024  0.000000     0.0  
              8     CSM5FZ44  0.000000  0.482759  0.000000  0.000000     0.0  
              9     CSM5FZ46  0.000000  0.495347  0.000005  0.000000     0.0  
...                                ...       ...       ...       ...     ...  
      2021    11    MSM5LLIO  0.000000  0.209143  0.000024  0.000000     0.0  
      2026    4     MSM5LLIQ  0.000000  0.406457  0.000003  0.000000     0.0  
      2027    4     MSM5LLIS  0.000000  0.000000  0.000000  0.000000     0.0  
      2014    9     MSM5ZOJY  0.000002  0.003325  0.006679  0.000000     0.0  
             

In [19]:
# I think we actually need to aggregate recursively, like this, to avoid missing reps

last_df = data["ibd"]
for x in [99, 97, 94, 91, 88, 85, 82, 79, 76]:
    otu2rep, rep2otus, tree = generate_otu_keys(x)
    df = aggregate_otu_table(last_df, otu2rep, rep2otus)
    summed = df.sum(axis=1)
    print(x, df.shape)
    print(x, "MIN", summed.min())
    print(x, "MEAN", summed.mean())
    print(x, "MAX", summed.max())
    print()


99 (98, 1370)
99 MIN 0.9999999999999979
99 MEAN 1.0
99 MAX 1.0000000000000024

97 (98, 1370)
97 MIN 0.9999999999999979
97 MEAN 1.0
97 MAX 1.0000000000000024

94 (98, 1171)
94 MIN 0.0
94 MEAN 0.5677273393344581
94 MAX 1.0

91 (98, 465)
91 MIN 0.0
91 MEAN 0.18160738967035886
91 MAX 0.6289697424356092

88 (98, 171)
88 MIN 0.0
88 MEAN 0.06683495371835792
88 MAX 0.5591397849462366

85 (98, 66)
85 MIN 0.0
85 MEAN 0.04223232836059484
85 MAX 0.5

82 (98, 27)
82 MIN 0.0
82 MEAN 0.037296837859774265
82 MAX 0.5

79 (98, 9)
79 MIN 0.0
79 MEAN 0.0334265372832717
79 MAX 0.5

76 (98, 1)
76 MIN 0.0
76 MEAN 5.471984406289964e-07
76 MAX 1.0171517208935678e-05



In [17]:
last_df.loc[('feces', 3001, 4, 'CSM5FZ3N')].sort_values(ascending=False).head(20)

589277     0.532276
328617     0.197485
583117     0.057012
157297     0.016904
186392     0.014421
370287     0.013384
195508     0.010486
564806     0.010472
701221     0.008948
320120     0.008133
514996     0.006435
367433     0.005508
909065     0.004740
3027156    0.004184
3148447    0.004117
186723     0.003957
355424     0.003609
363400     0.003531
4453343    0.002978
3327894    0.002912
Name: (feces, 3001, 4, CSM5FZ3N), dtype: float64

In [ ]:
last_df

229854    367523  200605  304382  290541  \
site  patient visit sample                                               
feces 3001    4     CSM5FZ3N     0.0  0.000000     0.0     0.0     0.0   
      3002    5     CSM5FZ3X     0.0  0.000003     0.0     0.0     0.0   
              6     CSM5FZ3Z     0.0  0.000000     0.0     0.0     0.0   
              8     CSM5FZ44     0.0  0.000000     0.0     0.0     0.0   
              9     CSM5FZ46     0.0  0.000000     0.0     0.0     0.0   
...                              ...       ...     ...     ...     ...   
      2021    11    MSM5LLIO     0.0  0.000000     0.0     0.0     0.0   
      2026    4     MSM5LLIQ     0.0  0.000000     0.0     0.0     0.0   
      2027    4     MSM5LLIS     0.0  0.000000     0.0     0.0     0.0   
      2014    9     MSM5ZOJY     0.0  0.000000     0.0     0.0     0.0   
              11    MSM633FF     0.0  0.000000     0.0     0.0     0.0   

                              529442    268769    530061  949904    726047  \
site  patient visit sample                                                   
feces 3001    4     CSM5FZ3N     0.0  0.000000  0.000000     0.0  0.000000   
      3002    5     CSM5FZ3X     0.0  0.000006  0.000033     0.0  0.000000   
              6     CSM5FZ3Z     0.0  0.000000  0.000000     0.0  0.000000   
              8     CSM5FZ44     0.0  0.000000  0.000000     0.0  0.000000   
              9     CSM5FZ46     0.0  0.000000  0.000010     0.0  0.000000   
...                              ...       ...       ...     ...       ...   
      2021    11    MSM5LLIO     0.0  0.000000  0.000000     0.0  0.000000   
      2026    4     MSM5LLIQ     0.0  0.000000  0.009461     0.0  0.000000   
      2027    4     MSM5LLIS     0.0  0.000000  0.000000     0.0  0.000000   
      2014    9     MSM5ZOJY     0.0  0.000000  0.000098     0.0  0.000013   
              11    MSM633FF     0.0  0.000645  0.000000     0.0  0.000000   

                              ...  162211   1117319   1523543    697063  \
site  patient visit sample    ...                                         
feces 3001    4     CSM5FZ3N  ...     0.0  0.000000  0.000019  0.000000   
      3002    5     CSM5FZ3X  ...     0.0  0.000045  0.000071  0.000003   
              6     CSM5FZ3Z  ...     0.0  0.000000  0.000004  0.000000   
              8     CSM5FZ44  ...     0.0  0.000000  0.000000  0.000000   
              9     CSM5FZ46  ...     0.0  0.000000  0.000000  0.000000   
...                           ...     ...       ...       ...       ...   
      2021    11    MSM5LLIO  ...     0.0  0.000000  0.000038  0.000000   
      2026    4     MSM5LLIQ  ...     0.0  0.000000  0.000044  0.000000   
      2027    4     MSM5LLIS  ...     0.0  0.000000  0.000000  0.000000   
      2014    9     MSM5ZOJY  ...     0.0  0.000000  0.000121  0.000002   
              11    MSM633FF  ...     0.0  0.000645  0.000081  0.000000   

                              4332943  1003894    760005    723019  250269  \
site  patient visit sample                                                   
feces 3001    4     CSM5FZ3N      0.0      0.0  0.000000  0.000000     0.0   
      3002    5     CSM5FZ3X      0.0      0.0  0.000000  0.000024     0.0   
              6     CSM5FZ3Z      0.0      0.0  0.000000  0.000000     0.0   
              8     CSM5FZ44      0.0      0.0  0.000000  0.000000     0.0   
              9     CSM5FZ46      0.0      0.0  0.000000  0.000000     0.0   
...                               ...      ...       ...       ...     ...   
      2021    11    MSM5LLIO      0.0      0.0  0.000000  0.000000     0.0   
      2026    4     MSM5LLIQ      0.0      0.0  0.000003  0.000000     0.0   
      2027    4     MSM5LLIS      0.0      0.0  0.000000  0.000000     0.0   
      2014    9     MSM5ZOJY      0.0      0.0  0.000006  0.000002     0.0   
              11    MSM633FF      0.0      0.0  0.000645  0.000000     0.0   

                              679616  
site  patient visit sa

In [ ]:
last_df

229854    367523  200605  304382  290541  \
site  patient visit sample                                               
feces 3001    4     CSM5FZ3N     0.0  0.000000     0.0     0.0     0.0   
      3002    5     CSM5FZ3X     0.0  0.000003     0.0     0.0     0.0   
              6     CSM5FZ3Z     0.0  0.000000     0.0     0.0     0.0   
              8     CSM5FZ44     0.0  0.000000     0.0     0.0     0.0   
              9     CSM5FZ46     0.0  0.000000     0.0     0.0     0.0   
...                              ...       ...     ...     ...     ...   
      2021    11    MSM5LLIO     0.0  0.000000     0.0     0.0     0.0   
      2026    4     MSM5LLIQ     0.0  0.000000     0.0     0.0     0.0   
      2027    4     MSM5LLIS     0.0  0.000000     0.0     0.0     0.0   
      2014    9     MSM5ZOJY     0.0  0.000000     0.0     0.0     0.0   
              11    MSM633FF     0.0  0.000000     0.0     0.0     0.0   

                              529442    268769    530061  949904    726047  \
site  patient visit sample                                                   
feces 3001    4     CSM5FZ3N     0.0  0.000000  0.000000     0.0  0.000000   
      3002    5     CSM5FZ3X     0.0  0.000006  0.000033     0.0  0.000000   
              6     CSM5FZ3Z     0.0  0.000000  0.000000     0.0  0.000000   
              8     CSM5FZ44     0.0  0.000000  0.000000     0.0  0.000000   
              9     CSM5FZ46     0.0  0.000000  0.000010     0.0  0.000000   
...                              ...       ...       ...     ...       ...   
      2021    11    MSM5LLIO     0.0  0.000000  0.000000     0.0  0.000000   
      2026    4     MSM5LLIQ     0.0  0.000000  0.009461     0.0  0.000000   
      2027    4     MSM5LLIS     0.0  0.000000  0.000000     0.0  0.000000   
      2014    9     MSM5ZOJY     0.0  0.000000  0.000098     0.0  0.000013   
              11    MSM633FF     0.0  0.000645  0.000000     0.0  0.000000   

                              ...  162211   1117319   1523543    697063  \
site  patient visit sample    ...                                         
feces 3001    4     CSM5FZ3N  ...     0.0  0.000000  0.000019  0.000000   
      3002    5     CSM5FZ3X  ...     0.0  0.000045  0.000071  0.000003   
              6     CSM5FZ3Z  ...     0.0  0.000000  0.000004  0.000000   
              8     CSM5FZ44  ...     0.0  0.000000  0.000000  0.000000   
              9     CSM5FZ46  ...     0.0  0.000000  0.000000  0.000000   
...                           ...     ...       ...       ...       ...   
      2021    11    MSM5LLIO  ...     0.0  0.000000  0.000038  0.000000   
      2026    4     MSM5LLIQ  ...     0.0  0.000000  0.000044  0.000000   
      2027    4     MSM5LLIS  ...     0.0  0.000000  0.000000  0.000000   
      2014    9     MSM5ZOJY  ...     0.0  0.000000  0.000121  0.000002   
              11    MSM633FF  ...     0.0  0.000645  0.000081  0.000000   

                              4332943  1003894    760005    723019  250269  \
site  patient visit sample                                                   
feces 3001    4     CSM5FZ3N      0.0      0.0  0.000000  0.000000     0.0   
      3002    5     CSM5FZ3X      0.0      0.0  0.000000  0.000024     0.0   
              6     CSM5FZ3Z      0.0      0.0  0.000000  0.000000     0.0   
              8     CSM5FZ44      0.0      0.0  0.000000  0.000000     0.0   
              9     CSM5FZ46      0.0      0.0  0.000000  0.000000     0.0   
...                               ...      ...       ...       ...     ...   
      2021    11    MSM5LLIO      0.0      0.0  0.000000  0.000000     0.0   
      2026    4     MSM5LLIQ      0.0      0.0  0.000003  0.000000     0.0   
      2027    4     MSM5LLIS      0.0      0.0  0.000000  0.000000     0.0   
      2014    9     MSM5ZOJY      0.0      0.0  0.000006  0.000002     0.0   
              11    MSM633FF      0.0      0.0  0.000645  0.000000     0.0   

                              679616  
site  patient visit sa

In [65]:
from src.greengenes import parse_similarity_map, merge_otu_table, combine_similarity_maps

# similarity_map = parse_similarity_map(91)
# clustered_otus = merge_otu_table(data["ibd"], similarity_map)
# clustered_otus.sum(axis=1).min()

combined_map = combine_similarity_maps(
    [parse_similarity_map(x) for x in [99, 97, 94, 91, 88, 85, 82, 79, 76]]
)
clustered_otus = merge_otu_table(data["ibd"], combined_map)
clustered_otus

229854    367523    200605    229982  304382  \
site  patient visit sample                                                     
feces 3001    4     CSM5FZ3N  0.000418  0.000014  0.000064  0.000006     0.0   
      3002    5     CSM5FZ3X  0.073538  0.000142  0.000315  0.000006     0.0   
              6     CSM5FZ3Z  0.008200  0.000004  0.000036  0.000166     0.0   
              8     CSM5FZ44  0.000000  0.000000  0.000000  0.000000     0.0   
              9     CSM5FZ46  0.015164  0.000005  0.000015  0.000030     0.0   
...                                ...       ...       ...       ...     ...   
      2021    11    MSM5LLIO  0.012636  0.007464  0.000283  0.000000     0.0   
      2026    4     MSM5LLIQ  0.016416  0.000036  0.014859  0.000104     0.0   
      2027    4     MSM5LLIS  0.000000  0.000000  0.000000  0.000000     0.0   
      2014    9     MSM5ZOJY  0.023822  0.000019  0.018818  0.008048     0.0   
              11    MSM633FF  0.001451  0.001854  0.023215  0.001612     0.0   

                               4406763     88848    104010   4446882  290541  \
site  patient visit sample                                                     
feces 3001    4     CSM5FZ3N  0.000000  0.000000  0.000138  0.000008     0.0   
      3002    5     CSM5FZ3X  0.000003  0.000000  0.001712  0.000110     0.0   
              6     CSM5FZ3Z  0.000000  0.000008  0.000182  0.000004     0.0   
              8     CSM5FZ44  0.000000  0.000000  0.000000  0.000000     0.0   
              9     CSM5FZ46  0.000000  0.000000  0.000020  0.000010     0.0   
...                                ...       ...       ...       ...     ...   
      2021    11    MSM5LLIO  0.000000  0.000000  0.016882  0.001227     0.0   
      2026    4     MSM5LLIQ  0.000000  0.000008  0.000239  0.000719     0.0   
      2027    4     MSM5LLIS  0.000000  0.000000  0.000000  0.000000     0.0   
      2014    9     MSM5ZOJY  0.000000  0.000050  0.031439  0.000144     0.0   
              11    MSM633FF  0.000242  0.000000  0.002499  0.002579     0.0   

                              ...   2185597   1599709   4332943  1003894  \
site  patient visit sample    ...                                          
feces 3001    4     CSM5FZ3N  ...  0.012994  0.000133  0.000122      0.0   
      3002    5     CSM5FZ3X  ...  0.015589  0.000258  0.166054      0.0   
              6     CSM5FZ3Z  ...  0.020735  0.000020  0.046214      0.0   
              8     CSM5FZ44  ...  0.007663  0.000000  0.114943      0.0   
              9     CSM5FZ46  ...  0.006736  0.000020  0.111767      0.0   
...                           ...       ...       ...       ...      ...   
      2021    11    MSM5LLIO  ...  0.014792  0.000088  0.000197      0.0   
      2026    4     MSM5LLIQ  ...  0.263067  0.001516  0.013659      0.0   
      2027    4     MSM5LLIS  ...  0.000000  0.000000  0.142857      0.0   
      2014    9     MSM5ZOJY  ...  0.148482  0.003670  0.000387      0.0   
              11    MSM633FF  ...  0.048525  0.001290  0.172981      0.0   

                               4387052    723019    250269    679616  \
site  patient visit sample                                             
feces 3001    4     CSM5FZ3N  0.000000  0.000000  0.000006  0.000000   
      3002    5     CSM5FZ3X  0.000000  0.000056  0.000006  0.000000   
              6     CSM5FZ3Z  0.000000  0.000000  0.000000  0.000000   
              8     CSM5FZ44  0.000000  0.000000  0.000000  0.000000   
              9     CSM5FZ46  0.000000  0.000000  0.000000  0.000000   
...                                ...       ...       ...       ...   
      2021    11    MSM5LLIO  0.000002  0.000000  0.000178  0.000000   
      2026    4     MSM5LLIQ  0.000005  0.000000  0.000000  0.000003   
      2027    4     MSM5LLIS  0.000000  0.000000  0.000000  0.000000   
      2014    9     MSM5ZOJY  0.000010  0.000006  0.000109  0.000000   
              11    MSM633FF  0.001290  0.000000  0.000000  0.000000   

                        

In [68]:
# Sanity check 1: sum to 1
# They do!!!
similarity_map99 = parse_similarity_map(99)
similarity_map = similarity_map99
for x in [97, 94, 91, 88, 85, 82, 79, 76, 73, 70, 67, 64, 61]:
    similarity_map = combine_similarity_maps(
        [similarity_map, parse_similarity_map(x)]
    )
    clustered_otus = merge_otu_table(data["ibd"], similarity_map)
    summed = clustered_otus.sum(axis=1)
    print(x, clustered_otus.shape, summed.min(), summed.mean(), summed.max())

97 (98, 1370) 0.9999999999999971 1.0000000000000002 1.000000000000008
94 (98, 1171) 0.9999999999999963 1.0000000000000002 1.0000000000000067
91 (98, 799) 0.9999999999999974 1.0 1.000000000000003
88 (98, 536) 0.9999999999999984 1.0 1.0000000000000033
85 (98, 337) 0.9999999999999994 1.0 1.0000000000000009
82 (98, 203) 0.9999999999999994 1.0 1.000000000000001
79 (98, 118) 0.9999999999999996 1.0 1.0000000000000007
76 (98, 82) 0.9999999999999996 1.0 1.0000000000000007
73 (98, 43) 0.9999999999999996 1.0 1.0000000000000004
70 (98, 24) 0.999999999999999 1.0 1.0000000000000029
67 (98, 13) 0.9999999999999974 1.0 1.0000000000000058
64 (98, 8) 0.9999999999999973 1.0 1.0000000000000064
61 (98, 6) 0.9999999999999973 1.0 1.0000000000000064
